In [ ]:
# load libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
from scipy import stats

import warnings

# suppress all warnings
warnings.filterwarnings('ignore')

# set seed
np.random.seed(42)

In [ ]:
# specify file paths
train_df = "train_subset.csv"
train_targets = "train_targets.csv"

# read in files
X = pd.read_csv(train_df)
y = pd.read_csv(train_targets)['AAC']   # keep only AAC column

print(X.shape)
print(y.shape)

(742, 457)
(742,)


Un-penalized Linear Regression Model

In [ ]:
# create dataframe to store results
model_df = pd.DataFrame(columns=['Model', 'Fold', 'Spearman', 'Pearson'])

# initialize the outer folds (5 folds, 80% train, 20% test)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# initialize variables to store best model correlation and features
best_corr = 0
best_fold = 0
best_feat = None

# loop through each of the outer five folds
fold = 1
for train_index, test_index in outer_cv.split(X):

  # split train and test
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # initialize linear regression model
  reg = linear_model.LinearRegression()

  # fit model
  reg.fit(X_train, y_train)

  # get predicted values for test data
  y_pred = reg.predict(X_test)

  # compute correlations
  s_cor = stats.spearmanr(y_pred, y_test)
  p_cor = stats.pearsonr(y_pred, y_test)

  # save model correlation and features (if better than previous)
  if s_cor[0] > best_corr:
          best_corr = s_cor[0]
          best_fold = fold
          best_feat = reg.coef_

  # save results to dataframe
  new_row = pd.DataFrame({'Model': ['Linear'], 'Fold': [fold], 'Spearman': [s_cor[0]], 'Pearson': [p_cor[0]]})
  model_df = pd.concat([model_df, new_row],ignore_index = True)

  # print results from fold
  print("Fold", fold, "Spearman correlation:", s_cor[0])

  fold += 1

# print best results
print("\nBest correlation:", best_corr, "from Fold", best_fold)

# create feature importance dataframe
feature_importance = pd.DataFrame({
    'Peak': X_train.columns,
    'Weight': best_feat
}).sort_values(by='Weight', ascending=False)

# save feature importance dataframe
filename = f"lm_features.csv"
feature_importance.to_csv(filename, index=False)

model_df.to_csv('lm.csv', index=False)

Fold 1 Spearman correlation: 0.16514979607280156
Fold 2 Spearman correlation: 0.37989986437942075
Fold 3 Spearman correlation: 0.26720022804599586
Fold 4 Spearman correlation: 0.33431679405580367
Fold 5 Spearman correlation: 0.24743599948493225

Best correlation: 0.37989986437942075 from Fold 2


LASSO Model

In [ ]:
# create dataframe to store results
model_df = pd.DataFrame(columns=['Model', 'Fold', 'Spearman', 'Pearson', 'alpha', 'max_iter'])

# initialize the outer folds (5 folds, 80% train, 20% test)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# initialize variables to store best model correlation and features
best_corr = 0
best_fold = 0
best_feat = None

# loop through each of the outer five folds
fold = 1
for train_index, test_index in outer_cv.split(X):

  # split train and test
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # initialize LASSO model
  lasso = linear_model.Lasso()

  # specify parameters for optimization
  parameters = {
      'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
      'max_iter': [500, 1000, 5000, 7500]
    }

  # identify optimal parameters
  reg = GridSearchCV(
      estimator = lasso,
      param_grid = parameters,
      #verbose=2
    )

  # fit model
  reg.fit(X_train, y_train)

  # get best model parameters
  reg_best = reg.best_estimator_

  alpha = reg.best_params_['alpha']
  max_iter = reg.best_params_['max_iter']

  # get predicted values for test data
  y_pred = reg_best.predict(X_test)

  # compute correlations
  s_cor = stats.spearmanr(y_pred, y_test)
  p_cor = stats.pearsonr(y_pred, y_test)

  # save model correlation and features (if better than previous)
  if s_cor[0] > best_corr:
          best_corr = s_cor[0]
          best_fold = fold
          best_feat = reg_best.coef_

  # save results to dataframe
  new_row = pd.DataFrame({'Model': ['LASSO'], 'Fold': [fold], 'Spearman': [s_cor[0]], 'Pearson': [p_cor[0]],
                          'alpha': [alpha], 'max_iter': [max_iter]})
  model_df = pd.concat([model_df, new_row],ignore_index = True)

  # print results from fold
  print("Fold", fold, "Spearman correlation:", s_cor[0])

  fold += 1

# print results
print("\nBest correlation:", best_corr, "from Fold", best_fold)

# create feature importance dataframe
feature_importance = pd.DataFrame({
    'Peak': X_train.columns,
    'Weight': best_feat
}).sort_values(by='Weight', ascending=False)

# save feature importance dataframe
filename = f"lasso_features.csv"
feature_importance.to_csv(filename, index=False)

model_df.to_csv('lasso.csv', index=False)

Fold 1 Spearman correlation: 0.38029056801264327
Fold 2 Spearman correlation: 0.30256149950651956
Fold 3 Spearman correlation: 0.4391997320041713
Fold 4 Spearman correlation: 0.42982662342267414
Fold 5 Spearman correlation: 0.4458852904947652

Best correlation: 0.4458852904947652 from Fold 5


Ridge Model

In [ ]:
# create dataframe to store results
model_df = pd.DataFrame(columns=['Model', 'Fold', 'Spearman', 'Pearson', 'alpha', 'max_iter'])

# initialize the outer folds (5 folds, 80% train, 20% test)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# initialize variables to store best model correlation and features
best_corr = 0
best_fold = 0
best_feat = None

# loop through each of the outer five folds
fold = 1
for train_index, test_index in outer_cv.split(X):

  # split train and test
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # initialize LASSO model
  ridge = linear_model.Ridge()

  # specify parameters for optimization
  parameters = {
      'alpha': [0.1, 1, 10, 100],
      'max_iter': [500, 1000, 5000, 7500]
    }

  # identify optimal parameters
  reg = GridSearchCV(
      estimator = ridge,
      param_grid = parameters,
      #verbose=2
    )

  # fit model
  reg.fit(X_train, y_train)

  # get best model parameters
  reg_best = reg.best_estimator_

  alpha = reg.best_params_['alpha']
  max_iter = reg.best_params_['max_iter']

  # get predicted values for test data
  y_pred = reg_best.predict(X_test)

  # compute correlations
  s_cor = stats.spearmanr(y_pred, y_test)
  p_cor = stats.pearsonr(y_pred, y_test)

  # save model correlation and features (if better than previous)
  if s_cor[0] > best_corr:
          best_corr = s_cor[0]
          best_fold = fold
          best_feat = reg_best.coef_

  # save results to dataframe
  new_row = pd.DataFrame({'Model': ['Ridge'], 'Fold': [fold], 'Spearman': [s_cor[0]], 'Pearson': [p_cor[0]],
                          'alpha': [alpha], 'max_iter': [max_iter]})
  model_df = pd.concat([model_df, new_row],ignore_index = True)

  # print results from fold
  print("Fold", fold, "Spearman correlation:", s_cor[0])

  fold += 1

# print results
print("\nBest correlation:", best_corr, "from Fold", best_fold)

# create feature importance dataframe
feature_importance = pd.DataFrame({
    'Peak': X_train.columns,
    'Weight': best_feat
}).sort_values(by='Weight', ascending=False)

# save feature importance dataframe
filename = f"ridge_features.csv"
feature_importance.to_csv(filename, index=False)

model_df.to_csv('ridge.csv', index=False)

Fold 1 Spearman correlation: 0.13362246836797476
Fold 2 Spearman correlation: 0.4032036886231177
Fold 3 Spearman correlation: 0.3181470711948323
Fold 4 Spearman correlation: 0.2812802511462029
Fold 5 Spearman correlation: 0.2400581932436511

Best correlation: 0.4032036886231177 from Fold 2


Elastic Net Model

In [ ]:
# create dataframe to store results
model_df = pd.DataFrame(columns=['Model', 'Fold', 'Spearman', 'Pearson', 'alpha', 'l1_ratio', 'max_iter'])

# initialize the outer folds (5 folds, 80% train, 20% test)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# initialize variables to store best model correlation and features
best_corr = 0
best_fold = 0
best_feat = None

# loop through each of the outer five folds
fold = 1
for train_index, test_index in outer_cv.split(X):

  # split train and test
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # initialize Elastic Net model
  en = linear_model.ElasticNet()

  # specify parameters for optimization
  parameters = {
    'alpha': [0.1, 1, 10, 100],
    'l1_ratio': [0.2, 0.5, 0.8],
    'max_iter': [1000, 5000, 7500]
  }

  # identify optimal parameters
  reg = GridSearchCV(
      estimator = en,
      param_grid = parameters,
      #verbose=2
    )

  # fit model
  reg.fit(X_train, y_train)

  # get best model parameters
  reg_best = reg.best_estimator_

  alpha = reg.best_params_['alpha']
  l1_ratio = reg.best_params_['l1_ratio']
  max_iter = reg.best_params_['max_iter']

  # get predicted values for test data
  y_pred = reg_best.predict(X_test)

  # compute correlations
  s_cor = stats.spearmanr(y_pred, y_test)
  p_cor = stats.pearsonr(y_pred, y_test)

  # save model correlation and features (if better than previous)
  if s_cor[0] > best_corr:
          best_corr = s_cor[0]
          best_fold = fold
          best_feat = reg_best.coef_

  # save results to dataframe
  new_row = pd.DataFrame({'Model': ['ElasticNet'], 'Fold': [fold], 'Spearman': [s_cor[0]], 'Pearson': [p_cor[0]], 'alpha': [alpha], 'l1_ratio': [l1_ratio], 'max_iter': [max_iter]})
  model_df = pd.concat([model_df, new_row],ignore_index = True)

  # print results from fold
  print("Fold", fold, "Spearman correlation:", s_cor[0])

  fold += 1

# print results
print("\nBest correlation:", best_corr, "from Fold", best_fold)

# create feature importance dataframe
feature_importance = pd.DataFrame({
    'Peak': X_train.columns,
    'Weight': best_feat
}).sort_values(by='Weight', ascending=False)

# save feature importance dataframe
filename = f"en_features.csv"
feature_importance.to_csv(filename, index=False)

model_df.to_csv('en.csv', index=False)

Fold 1 Spearman correlation: 0.34606416294605463
Fold 2 Spearman correlation: 0.39011320164331714
Fold 3 Spearman correlation: 0.526304489082618
Fold 4 Spearman correlation: 0.43093162852203315
Fold 5 Spearman correlation: 0.44804162372885764

Best correlation: 0.526304489082618 from Fold 3


Random Forest Model

In [ ]:
# create dataframe to store results
model_df = pd.DataFrame(columns=['Model', 'PSet', 'Fold', 'Spearman', 'Pearson', 'n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'max_features'])

# initialize the outer folds (5 folds, 80% train, 20% test)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# initialize variables to store best model correlation and features
best_corr = 0
best_fold = 0
best_feat = None

# loop through each of the outer five folds
fold = 1
for train_index, test_index in outer_cv.split(X):

  #print("Starting fold", fold)

  # split train and test
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # initialize Random Forest model
  rf = RandomForestRegressor()

  # specify parameters for optimization
  parameters = {
    'n_estimators': [10, 50, 100, 150, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2']
  }

  # identify optimal parameters
  reg = GridSearchCV(
      estimator = rf,
      param_grid = parameters,
      #verbose=2
    )

  # fit model
  reg.fit(X_train, y_train)

  # get best model parameters
  reg_best = reg.best_estimator_

  n_estimators = reg.best_params_['n_estimators']
  max_depth = reg.best_params_['max_depth']
  min_samples_split = reg.best_params_['min_samples_split']
  min_samples_leaf = reg.best_params_['min_samples_leaf']
  max_features = reg.best_params_['max_features']

  # get predicted values for test data
  y_pred = reg_best.predict(X_test)

  # compute correlations
  s_cor = stats.spearmanr(y_pred, y_test)
  p_cor = stats.pearsonr(y_pred, y_test)

  # save model correlation and features (if better than previous)
  if s_cor[0] > best_corr:
          best_corr = s_cor[0]
          best_fold = fold
          best_feat = reg_best.feature_importances_

  # save results to dataframe
  new_row = pd.DataFrame({'Model': ['Random Forest'], 'Fold': [fold], 'Spearman': [s_cor[0]], 'Pearson': [p_cor[0]],
                          'n_estimators': [n_estimators], 'max_depth': [max_depth], 'min_samples_split': [min_samples_split],
                          'min_samples_leaf': [min_samples_leaf], 'max_features': [max_features]})
  model_df = pd.concat([model_df, new_row],ignore_index = True)

  # print results from fold
  print("Fold", fold, "Spearman correlation:", s_cor[0])

  fold += 1

# print results
print("\nBest correlation:", best_corr, "from Fold", best_fold)

# create feature importance dataframe
feature_importance = pd.DataFrame({
    'Peak': X_train.columns,
    'Weight': best_feat
}).sort_values(by='Weight', ascending=False)

# save feature importance dataframe
filename = f"rf_features.csv"
feature_importance.to_csv(filename, index=False)

model_df.to_csv('rf.csv', index=False)

Fold 1 Spearman correlation: 0.44140435462049227
Fold 2 Spearman correlation: 0.43563239500489614
Fold 3 Spearman correlation: 0.4798294923660282
Fold 4 Spearman correlation: 0.522691474084379
Fold 5 Spearman correlation: 0.5292067470320231

Best correlation: 0.5292067470320231 from Fold 5


XGBoost Model

In [ ]:
# create dataframe to store results
model_df = pd.DataFrame(columns=['Model', 'PSet', 'Fold', 'Spearman', 'Pearson'])

# initialize the outer folds (5 folds, 80% train, 20% test)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# initialize variables to store best model correlation and features
best_corr = 0
best_fold = 0
best_feat = None

# loop through each of the outer five folds
fold = 1
for train_index, test_index in outer_cv.split(X):

  # split train and test
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  # initialize XGBoost model
  reg = xgb.XGBRegressor(tree_method="hist",
                        early_stopping_rounds=2,
                        eval_metric="rmse", verbosity=0,
                        objective='reg:squarederror',
                        max_depth=5, subsample=0.8)

  # fit model
  reg.fit(X_train, y_train, eval_set = [(X_test, y_test)], verbose=0)

  # get predicted values for test data
  y_pred = reg.predict(X_test)

  # compute correlations
  s_cor = stats.spearmanr(y_pred, y_test)
  p_cor = stats.pearsonr(y_pred, y_test)

  # save model correlation and features (if better than previous)
  if s_cor[0] > best_corr:
          best_corr = s_cor[0]
          best_fold = fold
          best_feat = reg.feature_importances_

  # save results to dataframe
  new_row = pd.DataFrame({'Model': ['Random Forest'], 'Fold': [fold], 'Spearman': [s_cor[0]], 'Pearson': [p_cor[0]]})
  model_df = pd.concat([model_df, new_row],ignore_index = True)

  # print results from fold
  print("Fold", fold, "Spearman correlation:", s_cor[0])

  fold += 1

# print results
print("\nBest correlation:", best_corr, "from Fold", best_fold)

# create feature importance dataframe
feature_importance = pd.DataFrame({
    'Peak': X_train.columns,
    'Weight': best_feat
}).sort_values(by='Weight', ascending=False)

# save feature importance dataframe
filename = f"xg_features.csv"
feature_importance.to_csv(filename, index=False)

model_df.to_csv('xg.csv', index=False)

Fold 1 Spearman correlation: 0.3242947460344039
Fold 2 Spearman correlation: 0.4268242632099374
Fold 3 Spearman correlation: 0.47536812919016647
Fold 4 Spearman correlation: 0.48367816860854423
Fold 5 Spearman correlation: 0.5317665829657139

Best correlation: 0.5317665829657139 from Fold 5
